In [1]:
!pip install git+https://github.com/remykarem/pandas-lightning#egg=pandas-lightning

  Cloning https://github.com/remykarem/pandas-lightning to /tmp/pip-install-vq2a_8y2/pandas-lightning_cbd520c224d5453d908c5449648ea3d9
  Running command git clone -q https://github.com/remykarem/pandas-lightning /tmp/pip-install-vq2a_8y2/pandas-lightning_cbd520c224d5453d908c5449648ea3d9
  Created wheel for pandas-lightning: filename=pandas_lightning-0.0.1rc0-py3-none-any.whl size=24443 sha256=e50de96999056b03d4a9ffee166da89a84cf13b885d2ed8bf48ab1876095c611
  Stored in directory: /tmp/pip-ephem-wheel-cache-qnv6zzcz/wheels/88/c8/60/df258e6b634ea9f2432284d8939a86858a7e833e824e0a2b90
Successfully built pandas-lightning


In [2]:
import os

import numpy as np
import pandas as pd
import pandas_lightning
from pandas_lightning import Pipeline

from sklearn.model_selection import train_test_split
import seaborn as sns

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

sns.set_theme()
sns.set(rc={'figure.figsize':(14.7,8.27)})

/kaggle/input/hdb-resale-price-prediction-full-train-set/train_edited.csv
/kaggle/input/cs5228-2020-semester-2-final-project/train.csv
/kaggle/input/cs5228-2020-semester-2-final-project/example-submission.csv
/kaggle/input/cs5228-2020-semester-2-final-project/test.csv
/kaggle/input/cs5228-2020-semester-2-final-project/auxiliary-data/sg-commerical-centres.csv
/kaggle/input/cs5228-2020-semester-2-final-project/auxiliary-data/sg-gov-markets-hawker-centres.csv
/kaggle/input/cs5228-2020-semester-2-final-project/auxiliary-data/sg-train-stations.csv
/kaggle/input/cs5228-2020-semester-2-final-project/auxiliary-data/sg-population-demographics.csv
/kaggle/input/cs5228-2020-semester-2-final-project/auxiliary-data/sg-shopping-malls.csv
/kaggle/input/cs5228-2020-semester-2-final-project/auxiliary-data/sg-primary-schools.csv
/kaggle/input/cs5228-2020-semester-2-final-project/auxiliary-data/sg-secondary-schools.csv


In [5]:
df_ = pd.read_csv("/kaggle/input/hdb-resale-price-prediction-full-train-set/train_edited.csv")

In [3]:
df_ = pd.read_csv("/kaggle/input/cs5228-2020-semester-2-final-project/train.csv")

In [4]:
def get_average_storey(storey):
    a, b = storey.str.split(" to ").str
    return (b.astype(int) + a.astype(int))/2

def get_flat_age(year, lease_commence_date):
    return year - lease_commence_date

def is_inauspicious(block):
    return block.isin(["13", "4", "44", "444"])

def is_before_covid(date):
    return date < "20200201"

def planning_area_to_district(planning_area):
    # https://www.ura.gov.sg/realEstateIIWeb/resources/misc/list_of_postal_districts.html
    # https://www.harrylau.com/basic-real-estate-knowledge-you-must-know/singapore-district-and-planning-area/
    
    # Note that ['clementi', 'kallang', 'bukit merah', 'outram', 'downtown core'] span multiple districts
    
    DISTRICT_MAPPING = {
        1: ["raffles place", "cecil", "marina", "people’s park"],
        2: ["anson", "tanjong pagar"],
        3: ["queenstown", "tiong bahru", "alexandra", "outram"], # outram is 3 based on wiki
        4: ["telok blangah", "harbourfront", "sentosa", "keppel", "mount faber", "bukit merah"], # bukit merah is 2, 3, 4, 5, 9, 10
        5: ["pasir panjang", "buona vista", "dover", "west coast", "clementi new town", "clementi"], # we place clementi together with clementi new town
        6: ["high street", "beach road", "city hall"],
        7: ["middle road", "golden mile", "bugis", "rochor", "downtown core"], # downtown core is 1, 6, 7
        8: ["little india", "farrer park", "serangoon road"],
        9: ["orchard", "cairnhill", "river valley"],
        10: ["ardmore", "bukit timah", "holland road", "tanglin"],
        11: ["watten estate", "newton", "novena", "thomson"],
        12: ["balestier", "toa payoh", "serangoon"],
        13: ["macpherson", "braddell", "potong pasir"],
        14: ["geylang", "paya lebar", "eunos", "kembangan", "kallang"], # kallang is in 1, 7, 12, 13, 14, 15
        15: ["katong", "joo chiat", "amber road", "marine parade", "tanjong rhu", "meyer"],
        16: ["bedok", "upper east coast", "eastwood", "kew drive", "chai chee", "siglap"],
        17: ["loyang", "changi"],
        18: ["tampines", "pasir ris", "simei"],
        19: ["serangoon garden", "hougang", "punggol", "sengkang"],
        20: ["bishan", "ang mo kio", "braddell"],
        21: ["upper bukit timah", "ulu pandan"],
        22: ["jurong", "boon lay", "tuas", "lakeside", "jurong west", "jurong east"],
        23: ["hillview", "dairy farm", "bukit panjang", "choa chu kang", "bukit batok"],
        24: ["lim chu kang"],
        25: ["kranji", "woodgrove", "woodlands", "sungei kadut"],
        26: ["upper thomson", "springleaf"],
        27: ["yishun", "sembawang", "admiralty"],
        28: ["seletar", "yio chu kang"]
    }
    return planning_area.map_categorical_binning(DISTRICT_MAPPING).astype("category")

def is_prime_district(district):
    # https://www.propertyguru.com.sg/property-guides/ccr-ocr-rcr-region-singapore-ura-map-21045
    return district.isin([9, 10, 11])

def is_core(prime_district, planning_area):
    # https://www.propertyguru.com.sg/property-guides/ccr-ocr-rcr-region-singapore-ura-map-21045
    return prime_district | planning_area.isin(["bugis", "city hall", "sentosa", "shenton way", "tanjong pagar",
                                                "boat quay", "raffles place", "marina downtown", "suntec city"]) 

def can_use_cpf(remaining_lease):
    return remaining_lease < 30

In [5]:
def process(data):
    
    return data.copy(

        ).rename(
            columns={"month": "date"}

        ).lambdas(inplace=True).sapply(
            flat_type=lambda s: s.str.replace("-", " "),

        ).lambdas(inplace=True).astype(
            planning_area="category",
            flat_model="category",
            subzone="category",
            region="category",
            town="category",
            date="datetime",
            flat_type=['1 room', '2 room', '3 room', '4 room', '5 room', 'executive', 'multi generation']

        ).lambdas(inplace=True).sapply(
            year=("date", lambda date: date.dt.year),
            avg_storey=("storey_range", get_average_storey),
            remaining_lease=(["year", "lease_commence_date"], get_flat_age),
            inauspicious=("block", is_inauspicious),
            before_covid=("date", is_before_covid),
            district=("planning_area", planning_area_to_district),
            prime_district=("district", is_prime_district),
            core_central_region=(["prime_district", "planning_area"], is_core),
            can_use_cpf=("remaining_lease", can_use_cpf),

        ).drop(
            columns=["storey_range", "eco_category", "elevation", "block", "lease_commence_date",
                     "latitude", "longitude",
                     "core_central_region", "can_use_cpf"] # 0 feature importance
        #              "markets_1km", "malls_1km", "comm_1km", "prisch_1km", "secsch_1km", "mrt_1km"]

        )

df = process(df_)
df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


,date,town,flat_type,street_name,floor_area_sqm,flat_model,subzone,planning_area,region,resale_price,year,avg_storey,remaining_lease,inauspicious,before_covid,district,prime_district
0,2001-08-01,pasir ris,4 room,pasir ris drive 4,118.0,model a,pasir ris drive,pasir ris,east region,209700.0,2001,2.0,12,False,True,18,False
1,2014-10-01,punggol,5 room,punggol field,110.0,improved,punggol field,punggol,north-east region,402300.0,2014,11.0,11,False,True,19,False
2,2020-09-01,sengkang,5 room,fernvale lane,112.0,premium apartment,fernvale,sengkang,north-east region,351000.0,2020,2.0,16,False,False,19,False
3,2000-10-01,clementi,3 room,clementi avenue 4,67.0,new generation,clementi north,clementi,west region,151200.0,2000,8.0,20,False,True,5,False
4,2013-01-01,bukit batok,3 room,bukit batok street 11,73.0,model a,bukit batok west,bukit batok,west region,318600.0,2013,8.0,28,False,True,23,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431727,2005-03-01,woodlands,4 room,Woodlands Drive 16,101.0,model a,woodlands south,woodlands,north region,238500.0,2005,2.0,5,False,True,25,False
431728,2016-04-01,sengkang,4 room,fernvale road,95.0,premium apartment,fernvale,sengkang,north-east region,376200.0,2016,14.0,4,False,True,19,False
431729,2011-01-01,tampines,3 room,tampines street 81,67.0,new generation,tampines west,tampines,east region,255600.0,2011,2.0,25,False,True,18,False
431730,2013-05-01,sengkang,5 room,compassvale walk,123.0,improved,sengkang town centre,sengkang,north-east region,508500.0,2013,17.0,14,False,True,19,False


In [12]:
submission_pipeline = Pipeline()
submission_pipeline.add(process)

In [ ]:
df.tests.info(pctg=False).sort_values(by="dtype")

In [ ]:
df.quickplot(
    numerical=["floor_area_sqm"],
    categorical=["flat_model"]

).ridgeplot()

# Modelling

* Drop nominal features
* One-hot nominal features
* Keep (label-encode) nominal features

In [13]:
from sklearn.experimental import enable_hist_gradient_boosting #for HistGBR # noqa
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_graphviz
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRFRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error

def rmse(clf, X_test, y_true):
    return np.sqrt(mean_squared_error(y_true, clf.predict(X_test)))

## Models for dropping nominal features

* Linear Regression
* Decision Tree
* Random Forest
* XGBRegressor
* GradientBoostingRegressor

RMSE: 50k-55k

In [8]:
X, y, metadata = df.dataset(pipelines=[submission_pipeline]).to_X_y(
    target="resale_price",
    nominal="drop",
    nans="remove",
    nominal_max_cardinality=100
)

X

/opt/conda/lib/python3.7/site-packages/pandas_lightning/dataframe_dataset.py:108: UserWarning: Found `object` and/or `datetime` categories. These categories are removed.
  warnings.warn("Found `object` and/or `datetime` categories. "


,flat_type,floor_area_sqm,year,avg_storey,remaining_lease,inauspicious,before_covid,prime_district
0,3,118.0,2001,2.0,12,0,1,0
1,4,110.0,2014,11.0,11,0,1,0
2,4,112.0,2020,2.0,16,0,0,0
3,2,67.0,2000,8.0,20,0,1,0
4,2,73.0,2013,8.0,28,0,1,0
...,...,...,...,...,...,...,...,...
431727,3,101.0,2005,2.0,5,0,1,0
431728,3,95.0,2016,14.0,4,0,1,0
431729,2,67.0,2011,2.0,25,0,1,0
431730,4,123.0,2013,17.0,14,0,1,0


In [34]:
def transform(data):

    # Drop
    data = data.drop(columns=metadata["removed"]["col_names"])

    # Ordinal
    for col_name in metadata["ordinal"]["col_names"]:
        d = {v:k for k,v in metadata["ordinal"]["label_mappings"][col_name].items()}
        data[col_name] = data[col_name].cat.rename_categories(d)
        
    # Nominal
    if metadata["nominal"]["one-hot"]:
        data = pd.get_dummies(data, columns=metadata["nominal"]["col_names"])
    else:
        for col_name in metadata["nominal"]["col_names"]:
            if col_name not in metadata["nominal"]["label_mappings"]:
                continue
            d = {v:k for k,v in metadata["nominal"]["label_mappings"][col_name].items()}
            data[col_name] = data[col_name].cat.rename_categories(d)
            
    # Bool
    for col_name in metadata["bool"]["col_names"]:
        data[col_name] = data[col_name].astype(int)

    return data

transform(test3)

,town,flat_type,floor_area_sqm,flat_model,planning_area,region,year,avg_storey,remaining_lease,inauspicious,before_covid,district,prime_district
0,4,3,94.0,3,4,2,2004,5.0,15,0,1,23,0
1,14,4,122.0,1,14,0,2001,5.0,4,0,1,18,0
2,10,2,67.0,3,10,2,2002,2.0,20,0,1,22,0
3,21,2,82.0,3,21,1,2015,5.0,34,0,1,20,0
4,3,4,117.0,6,3,2,2004,2.0,26,0,1,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107929,12,4,110.0,1,12,1,2008,11.0,5,0,1,19,0
107930,11,3,102.0,0,11,3,2006,14.0,7,1,1,14,0
107931,11,2,68.0,1,31,3,2000,8.0,21,0,1,7,0
107932,5,3,104.0,0,5,2,2009,11.0,21,0,1,22,0


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5228)

Linear Regression

In [31]:
model = LinearRegression()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, coef in zip(X.columns, model.coef_):
    print(f"{feat:20}: {coef}")

71242.3876721421
flat_type           : 32997.03616011075
floor_area_sqm      : 1660.8528842790276
year                : 12365.604026217063
avg_storey          : 4765.66803201816
remaining_lease     : -285.0781899186934
inauspicious        : 28096.465255520463
before_covid        : 42139.2678640087
prime_district      : 49364.44278401007


Decision Tree

In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

Random Forest

In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

XGBRegressor

In [ ]:
model = XGBRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

## Models for one-hot nominal features

* Linear Regression
* Decision Tree
* Random Forest
* XGBRegressor
* GradientBoostingRegressor

RMSE: 26k-28k

In [ ]:
X, y, metadata = df.dataset.to_X_y(
    target="resale_price",
    nominal="one-hot",
    nans="remove",
    nominal_max_cardinality=100
)

X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5228)

Linear Regression

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, coef in zip(X.columns, model.coef_):
    print(f"{feat:20}: {coef}")

Decision Tree

In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

Random Forest

In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

XGBRegressor

In [ ]:
model = XGBRegressor(n_estimators=10_000)
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

XGBRFRegressor

In [ ]:
model = XGBRFRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

## Models for nominal feature-aware (tree-based)

* HistGradientBoostingRegressor
* CatBoost
* LGBMRegressor

RMSE: 19k-21k

In [14]:
X, y, metadata = df.dataset(pipelines=[submission_pipeline]).to_X_y(
    target="resale_price",
    nominal="label",
    nans="remove",
    nominal_max_cardinality=200
)

X

/opt/conda/lib/python3.7/site-packages/pandas_lightning/dataframe_dataset.py:108: UserWarning: Found `object` and/or `datetime` categories. These categories are removed.
  warnings.warn("Found `object` and/or `datetime` categories. "


,floor_area_sqm,year,avg_storey,remaining_lease,flat_type,inauspicious,before_covid,prime_district,town,flat_model,subzone,planning_area,region,district
0,118.0,2001,2.0,12,3,0,1,0,0,0,0,0,0,18
1,110.0,2014,11.0,11,4,0,1,0,1,1,1,1,1,19
2,112.0,2020,2.0,16,4,0,0,0,2,2,2,2,1,19
3,67.0,2000,8.0,20,2,0,1,0,3,3,3,3,2,5
4,73.0,2013,8.0,28,2,0,1,0,4,0,4,4,2,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431727,101.0,2005,2.0,5,3,0,1,0,13,0,45,13,4,25
431728,95.0,2016,14.0,4,3,0,1,0,2,2,2,2,1,19
431729,67.0,2011,2.0,25,2,0,1,0,14,3,17,14,0,18
431730,123.0,2013,17.0,14,4,0,1,0,2,1,64,2,1,19


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5228)

In [25]:
hgb = HistGradientBoostingRegressor(max_iter=20_000, 
                                    categorical_features=metadata["nominal"]["col_indices"])
hgb.fit(X_train, y_train)
score = rmse(hgb, X_test, y_test)
print(score)

19458.59572018724


In [ ]:
cbr = CatBoostRegressor(cat_features=metadata["nominal"]["col_indices"], verbose=0)
cbr.fit(X_train, y_train)
score = rmse(cbr, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, cbr.feature_importances_):
    print(f"{feat:20}: {imp}")

In [23]:
lgr = LGBMRegressor(n_estimators=20_000)
lgr.fit(X_train, y_train, categorical_feature=metadata["nominal"]["col_indices"])
score = rmse(lgr, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, lgr.feature_importances_):
    print(f"{feat:20}: {imp}")

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


19448.733213599637
floor_area_sqm      : 159809
year                : 123817
avg_storey          : 99482
remaining_lease     : 138004
flat_type           : 4055
inauspicious        : 1402
before_covid        : 3971
prime_district      : 426
town                : 5855
flat_model          : 11968
subzone             : 48934
planning_area       : 788
region              : 359
district            : 1130


In [26]:
class Ensemble:
    
    def __init__(self, weights: list, *models):
        self.models = models
        self.weights = weights
        if len(self.models) != len(self.weights):
            raise ValueError("No. of weights and models must be the same")
        if sum(self.weights) != 1:
            raise ValueError("Weights must sum to 1")
            
    def predict(self, X):
        y = [weight*model.predict(X) 
             for weight, model in zip(self.weights, self.models)]
        y = np.vstack(y)
        mean = y.sum(axis=0)
        return mean

In [27]:
ens = Ensemble([0.5, 0.5], hgb, lgr)

In [28]:
rmse(ens, X_test, y_test)

18969.24662393572

# Next steps

Feature engineering
- [ ] Incorporate latlongs (train_edited.csv)

Modelling
* Random Forest
* Hyperparameter tuning with graph
* Ensemble prediction

In [ ]:
grid_search_grid = {
    'n_estimators': [10,100,1000,10000],
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
}

# Submission

In [15]:
test_ = pd.read_csv("/kaggle/input/cs5228-2020-semester-2-final-project/test.csv")

In [16]:
test = submission_pipeline(test_)
test

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


,floor_area_sqm,year,avg_storey,remaining_lease,flat_type,inauspicious,before_covid,prime_district,town,flat_model,subzone,planning_area,region,district
0,94.0,2004,5.0,15,3,0,1,0,3,11,22,3,4,23
1,122.0,2001,5.0,4,4,0,1,0,22,4,118,27,1,18
2,67.0,2002,2.0,20,2,0,1,0,12,11,131,13,4,22
3,82.0,2015,5.0,34,2,0,1,0,0,11,146,0,3,20
4,117.0,2004,2.0,26,4,0,1,0,9,16,34,9,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107929,110.0,2008,11.0,5,4,0,1,0,11,4,133,12,3,19
107930,102.0,2006,14.0,7,3,1,1,0,14,7,14,15,0,14
107931,68.0,2000,8.0,21,2,0,1,0,14,4,32,10,0,7
107932,104.0,2009,11.0,21,3,0,1,0,13,7,154,14,4,22


In [ ]:
preds = ens.predict(test)

In [ ]:
submission = pd.DataFrame({"Predicted": preds})
submission.to_csv("submission.csv", index_label="Id")

<a href="submission.csv">now</a>